In [1]:
import torch
import torch.nn as nn
from torchvision import transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from Dataset import Dataset
from models import Models



In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

test_dataset = Dataset(is_validation=True, device=device)

batch_size = 32
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

In [3]:
model_choice = 'resnet50'  
num_classes = 2  
dropout = 0  
model = Models(model_choice, num_out_classes=num_classes, dropout=dropout)


model.to(device)
model.eval()

c:\Users\Sadman\anaconda3\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
c:\Users\Sadman\anaconda3\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Models(
  (model): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
          (0)

In [ ]:
# Set hyperparameters
num_epochs = 10
learning_rate = 0.1


# Load the ImageNet Object Localization Challenge dataset
train_dataset = Dataset(is_validation=False, device=device)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)

# Parallelize training across multiple GPUs
model = torch.nn.DataParallel(model)

# Set the model to run on the device
model = model.to(device)

# Define the loss function and optimizer
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Train the model...
for epoch in range(num_epochs):
    for inputs, labels in train_loader:
        # Move input and label tensors to the device
        inputs = inputs.to(device)
        labels = labels.to(device)

        # Zero out the optimizer
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward pass
        loss.backward()
        optimizer.step()

    # Print the loss for every epoch
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item():.4f}')

print(f'Finished Training, Loss: {loss.item():.4f}')

In [19]:
criterion = nn.CrossEntropyLoss()

test_loss = 0.0
correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_dataloader:
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        #print(outputs.data.shape)
        #print(labels.shape)
        loss = criterion(outputs, labels)
        test_loss += loss.item()

        # Get predicted labels
        _, predicted = torch.max(outputs.data, 1)
        #print(_.shape)
        #print(predicted.shape)
        total += labels.size(0)
        correct += (predicted == labels[:, 0]).sum().item()

# Calculate average loss and accuracy
average_loss = test_loss / len(test_dataloader)
accuracy = correct / total * 100

print(f"Average Loss: {average_loss:.4f}")
print(f"Accuracy: {accuracy:.2f}%")

torch.Size([32, 2])
torch.Size([32, 2])
torch.Size([32])
torch.Size([32])
torch.Size([32, 2])
torch.Size([32, 2])
torch.Size([32])
torch.Size([32])
torch.Size([32, 2])
torch.Size([32, 2])
torch.Size([32])
torch.Size([32])
torch.Size([32, 2])
torch.Size([32, 2])
torch.Size([32])
torch.Size([32])
torch.Size([32, 2])
torch.Size([32, 2])
torch.Size([32])
torch.Size([32])


KeyboardInterrupt: 